# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

dbo.annual_2016(id, series_id, year, period, value, footnote_codes, original_file)

dbo.datatype(data_type_code, data_type_text)

dbo.footnote(footnote_code, footnote_text)

dbo.industry(id, industry_code, naics_code, publishing_status, industry_name, display_level, selectable, sort_sequence)

dbo.january_2017(id, series_id, year, period, value, footnote_codes, original_file)

dbo.period(period_code, month_abbr, month)

dbo.seasonal(industry_code, seasonal_text)

dbo.series(series_id, supersector_code, industry_code, data_type_code, seasonal, series_title)

dbo.supersector(supersector_code, supersector_name)

2. What is the datatype for women employees?

In [ ]:
SELECT data_type_code
FROM dbo.datatype
WHERE data_type_text = 'WOMEN EMPLOYEES';

-- datatype for 'WOMEN EMPLOYEES' is 10

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
SELECT series_id
FROM dbo.series
WHERE supersector_code = (
    SELECT supersector_code
    FROM dbo.supersector
    WHERE supersector_name = 'Financial activities'
) AND industry_code = (
    SELECT industry_code
    FROM dbo.industry
    WHERE industry_name = 'Commercial banking'
) AND data_type_code = (
    SELECT data_type_code
    FROM dbo.datatype
    WHERE data_type_text = 'WOMEN EMPLOYEES'
);

-- The series_id for women employees in the commercial banking industry in the
-- financial activities supersector is 'CEU5552211010'.
 

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
SELECT 
    ROUND(SUM(an.value),0) AS total_employees_2016
FROM dbo.annual_2016 AS an
JOIN dbo.series AS se 
    ON an.series_id = se.series_id
JOIN dbo.datatype AS dt 
    ON se.data_type_code = dt.data_type_code
WHERE dt.data_type_text = 'ALL EMPLOYEES';

-- Total number of all employees reported in 2016 across all industries, rounded to
-- the nearest whole number is: 2,340,612.

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:
SELECT 
    ROUND(SUM(an.value),0) AS total_women_employees_2016
FROM dbo.annual_2016 AS an
JOIN dbo.series AS se 
    ON an.series_id = se.series_id
JOIN dbo.datatype AS dt 
    ON se.data_type_code = dt.data_type_code
WHERE dt.data_type_text = 'WOMEN EMPLOYEES';

-- Total number of women employees reported in 2016 across all industries, rounded to
-- the nearest whole number is: 1,125,490.

3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:
SELECT 
    ROUND(SUM(an.value),0) AS grunt_employees
FROM dbo.annual_2016 AS an
JOIN dbo.series AS se 
    ON an.series_id = se.series_id
JOIN dbo.datatype AS dt 
    ON se.data_type_code = dt.data_type_code
WHERE dt.data_type_text = 'PRODUCTION AND NONSUPERVISORY EMPLOYEES';

-- Total number of production/nonsupervisory employees reported in 2016 across all industries, 
-- rounded to the nearest whole number is: 1,263,650.

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
SELECT 
    ROUND(AVG(jan.value),0) AS avg_grunt_wk_hours_jan2017
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se 
    ON jan.series_id = se.series_id
JOIN dbo.datatype AS dt 
    ON se.data_type_code = dt.data_type_code
WHERE dt.data_type_text LIKE 'AVERAGE WEEKLY HOURS%PRODUCTION AND NONSUPERVISORY%';

-- The average weekly hours worked by production and nonsupervisory employees across all industries,
-- excluding overtime, is: 36.

WITH avg_grunt_wk_hours_jan2017 AS (
    SELECT 
        ROUND(AVG(jan.value),0) AS avg_wk_hours_jan2017
    FROM dbo.january_2017 AS jan
    JOIN dbo.series AS se 
        ON jan.series_id = se.series_id
    JOIN dbo.datatype AS dt 
        ON se.data_type_code = dt.data_type_code
    WHERE dt.data_type_text LIKE 'AVERAGE WEEKLY HOURS%PRODUCTION AND NONSUPERVISORY%'),
avg_grunt_ot_hours_jan2017 AS (
    SELECT 
        ROUND(AVG(jan.value),0) AS avg_ot_hours_jan2017
    FROM dbo.january_2017 AS jan
    JOIN dbo.series AS se 
        ON jan.series_id = se.series_id
    JOIN dbo.datatype AS dt 
        ON se.data_type_code = dt.data_type_code
    WHERE dt.data_type_text LIKE 'AVERAGE WEEKLY OVERTIME%PRODUCTION AND NONSUPERVISORY%')
SELECT a.avg_wk_hours_jan2017 + b.avg_ot_hours_jan2017 AS total_avg_hours
FROM avg_grunt_wk_hours_jan2017 AS a
CROSS JOIN avg_grunt_ot_hours_jan2017 AS b;

-- If we sum average work hours and average overtime hours, that weekly average increases to: 40.

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:
SELECT 
    ROUND(SUM(jan.value), 2) AS total_wk_pay_grunts_jan2017
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
JOIN dbo.datatype AS dt
    ON se.data_type_code = dt.data_type_code
WHERE dt.data_type_code = (
    SELECT dt.data_type_code
    FROM dbo.datatype AS dt
    WHERE data_type_text LIKE '%PRODUCTION AND NONSUPERVISORY%'
      AND data_type_text LIKE 'AGGREGATE WEEKLY PAYROLLS%' 
);

-- The total weekly payroll for production and nonsupervisory employees across all industries
-- in January 2017, rounded to the nearest penny is $1,838,853,220.00.

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
WITH code_avg_wk_hours_grunts AS (
    SELECT data_type_code
    FROM datatype AS dt
    WHERE data_type_text LIKE '%PRODUCTION AND NONSUPERVISORY%'
        AND data_type_text LIKE 'AVERAGE WEEKLY HOURS%'
)
SELECT DISTINCT
    ind.industry_name,
    ind.naics_code,
    jan.value AS minmax_grunts_avg_hours_jan2017
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
JOIN dbo.datatype AS dt
    ON se.data_type_code = dt.data_type_code
JOIN dbo.industry AS ind
    ON se.industry_code = ind.industry_code
WHERE dt.data_type_code IN (
    SELECT data_type_code 
    FROM code_avg_wk_hours_grunts)
  AND (jan.value = (SELECT MAX(value) 
                    FROM dbo.january_2017 AS jan_max
                    JOIN dbo.series AS se_max ON jan_max.series_id = se_max.series_id
                    JOIN dbo.datatype AS dt_max ON se_max.data_type_code = dt_max.data_type_code
                    WHERE dt_max.data_type_code IN (
                        SELECT data_type_code 
                        FROM code_avg_wk_hours_grunts))
       OR jan.value = (SELECT MIN(value) 
                       FROM dbo.january_2017 AS jan_min
                       JOIN dbo.series AS se_min ON jan_min.series_id = se_min.series_id
                       JOIN dbo.datatype AS dt_min ON se_min.data_type_code = dt_min.data_type_code
                       WHERE dt_min.data_type_code IN (
                        SELECT data_type_code 
                        FROM code_avg_wk_hours_grunts)));

-- The industry with the highest average weekly hours worked by production and nonsupervisory
-- employees was 'Motor vehicle power train components' with 49.8 hours/weekly.
--
-- The industry with the lowest average weekly hours worked by production and nonsupervisory
-- employees was 'Fitness and recreational sports centers'with 16.7 hours/weekly.

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
-- Completing under the assumption that we are using January 2017 data based upon Slack comments.

WITH code_wk_payroll_grunts AS (
    SELECT data_type_code
    FROM datatype AS dt
    WHERE data_type_text LIKE '%PRODUCTION AND NONSUPERVISORY%'
      AND data_type_text LIKE 'AGGREGATE WEEKLY PAYROLLS%' 
)
SELECT DISTINCT
    ind.industry_name,
    ind.naics_code,
    jan.value AS minmax_grunts_payroll_jan2017
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
JOIN dbo.datatype AS dt
    ON se.data_type_code = dt.data_type_code
JOIN dbo.industry AS ind
    ON se.industry_code = ind.industry_code
WHERE dt.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
  AND ind.industry_name <> 'Total private'
  AND (jan.value = (SELECT MAX(value) 
                    FROM dbo.january_2017 AS jan_max
                    JOIN dbo.series AS se_max ON jan_max.series_id = se_max.series_id
                    JOIN dbo.datatype AS dt_max ON se_max.data_type_code = dt_max.data_type_code
                    JOIN dbo.industry AS ind_max ON se_max.industry_code = ind_max.industry_code
                    WHERE dt_max.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
                      AND ind_max.industry_name <> 'Total private')
       OR jan.value = (SELECT MIN(value) 
                       FROM dbo.january_2017 AS jan_min
                       JOIN dbo.series AS se_min ON jan_min.series_id = se_min.series_id
                       JOIN dbo.datatype AS dt_min ON se_min.data_type_code = dt_min.data_type_code
                       JOIN dbo.industry AS ind_min ON se_min.industry_code = ind_min.industry_code
                       WHERE dt_min.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
                         AND ind_min.industry_name <> 'Total private'));

-- The industry with the highest total weekly payroll for production and nonsupervisory
-- employees was 'Private service-providing'with $60,914,722.00
--
-- The industry with the lowest total weekly payroll for production and nonsupervisory
-- employees was 'Coin-operated industries and drycleaners' with $10,079.

-- Note: 'Total private' was excluded from the results as it appears that it may be an aggregate value
--        that overlaps or double-reports what may be in other industries. 
--
--        That amount was: $74,498,171.00, calculated by removing "AND ind.industry_name <> 'Total private'"
--        from the search query.

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Utilizing LEFT JOIN, as only data from annual_2016 was requested.

SELECT TOP 50
    an.*,
    se.series_title,
    se.industry_code,
    se.data_type_code
FROM dbo.annual_2016 AS an
LEFT JOIN dbo.series AS se
    ON an.series_id = se.series_id
ORDER BY id;

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting results to include only rows where data exists in both tables using INNER JOIN.

SELECT TOP 50
    se.series_id,
    se.series_title,
    se.data_type_code,
    dt.data_type_text
FROM dbo.series AS se
INNER JOIN dbo.datatype AS dt
    ON se.data_type_code = dt.data_type_code
ORDER BY se.series_id;

-- Sorting by series_id as 'id' does not exist in either table.

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting results to include only rows where data exists in both tables using INNER JOIN.

SELECT TOP 50
    se.series_id,
    se.series_title,
    se.industry_code,
    ind.industry_name,
    ind.naics_code,
    ind.display_level
FROM dbo.series AS se
INNER JOIN dbo.industry AS ind
    ON se.industry_code = ind.industry_code
ORDER BY ind.id

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
-- Using CROSS JOIN since avg_annual_payroll includes only one row, 
-- which is the average annual payroll value where data_type_code = 82.

WITH avg_annual_payroll AS (
    SELECT 
        AVG(an.value) AS avg_value_2016
    FROM dbo.annual_2016 AS an
    JOIN dbo.series AS se_avg
        ON an.series_id = se_avg.series_id
    WHERE se_avg.data_type_code = 82
)
SELECT 
    jan.series_id,
    se.industry_code,
    ind.industry_name,
    jan.value
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
JOIN dbo.industry AS ind
    ON se.industry_code = ind.industry_code
CROSS JOIN avg_annual_payroll 
WHERE se.data_type_code = 82
  AND jan.value > avg_annual_payroll.avg_value_2016
ORDER BY "value";

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- See above.

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
SELECT 
    ROUND(AVG(an.value), 2) AS avg_wk_earnings,
    an.year,
    an.period
FROM dbo.annual_2016 AS an
JOIN dbo.series AS se
    ON an.series_id = se.series_id
WHERE se.data_type_code = 30 
GROUP BY an.year, an.period

UNION

SELECT 
    ROUND(AVG(jan.value), 2) AS avg_wk_earnings,
    jan.year,
    jan.period
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
WHERE se.data_type_code = 30
GROUP BY jan.year, jan.period;

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

Production and nonsupervisory employees fared better in period M01 of 2017 with average weekly earnings equal to $808.53 versus $797.20 of period M13 in 2016.

2. In which industries did production and nonsupervisory employees fare better?

Private sector employees appeared to fare the best. Excluding anything mentioning 'private', 'Professional and business services' employees still saw eight figure values.

In [ ]:
WITH code_wk_payroll_grunts AS (
    SELECT data_type_code
    FROM datatype AS dt
    WHERE data_type_text LIKE '%PRODUCTION AND NONSUPERVISORY%'
      AND data_type_text LIKE 'AGGREGATE WEEKLY PAYROLLS%' 
)
SELECT DISTINCT
    ind.industry_name,
    ind.naics_code,
    jan.value AS minmax_grunts_payroll_jan2017
FROM dbo.january_2017 AS jan
JOIN dbo.series AS se
    ON jan.series_id = se.series_id
JOIN dbo.datatype AS dt
    ON se.data_type_code = dt.data_type_code
JOIN dbo.industry AS ind
    ON se.industry_code = ind.industry_code
WHERE dt.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
  AND ind.industry_name NOT LIKE '%PRIVATE%'
  AND (jan.value = (SELECT MAX(value) 
                    FROM dbo.january_2017 AS jan_max
                    JOIN dbo.series AS se_max ON jan_max.series_id = se_max.series_id
                    JOIN dbo.datatype AS dt_max ON se_max.data_type_code = dt_max.data_type_code
                    JOIN dbo.industry AS ind_max ON se_max.industry_code = ind_max.industry_code
                    WHERE dt_max.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
                      AND ind_max.industry_name NOT LIKE '%PRIVATE%')
       OR jan.value = (SELECT MIN(value) 
                       FROM dbo.january_2017 AS jan_min
                       JOIN dbo.series AS se_min ON jan_min.series_id = se_min.series_id
                       JOIN dbo.datatype AS dt_min ON se_min.data_type_code = dt_min.data_type_code
                       JOIN dbo.industry AS ind_min ON se_min.industry_code = ind_min.industry_code
                       WHERE dt_min.data_type_code IN (SELECT data_type_code FROM code_wk_payroll_grunts)
                         AND ind_min.industry_name NOT LIKE '%PRIVATE%'));

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

I would want to know the number of employees per industry. I think that could add another layer to the analysis.